In [1]:
import tensorflow as tf
import numpy as np

# lets take optimizers and activation functions as already given

class SequentialTFModel:
    def __init__(self, input_dim):
        self.layers = []
        self.input_dim = input_dim
        self.prev_dim = input_dim

    def add(self, units, activation='relu'):
        # Weight and bias initialization
        W = tf.Variable(tf.random.normal([self.prev_dim, units], stddev=0.1), trainable=True)
        b = tf.Variable(tf.zeros([units]), trainable=True)
        self.layers.append({'W': W, 'b': b, 'activation': activation})
        self.prev_dim = units

    def forward(self, X):
        out = X
        for layer in self.layers:
            Z = tf.matmul(out, layer['W']) + layer['b']
            if layer['activation'] == 'relu':
                out = tf.nn.relu(Z)
            elif layer['activation'] == 'sigmoid':
                out = tf.nn.sigmoid(Z)
            else:
                raise ValueError("Unsupported activation")
        return out

    def train(self, X, Y, epochs=100, lr=0.01, loss_fn='mse'):
        optimizer = tf.optimizers.Adam(lr)

        for epoch in range(epochs):
            with tf.GradientTape() as tape:
                predictions = self.forward(X)
                if loss_fn == 'mse':
                    loss = tf.reduce_mean((predictions - Y) ** 2)
                elif loss_fn == 'bce':
                    loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(Y, predictions))
                else:
                    raise ValueError("Unsupported loss")

            # Collect trainable variables
            variables = []
            for layer in self.layers:
                variables += [layer['W'], layer['b']]

            grads = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(grads, variables))

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")

    def predict(self, X):
        return self.forward(X)

In [11]:
X = tf.convert_to_tensor(np.random.randn(100, 2).astype(np.float32))
Y = tf.convert_to_tensor((np.sum(X.numpy(), axis=1) > 0).astype(np.float32).reshape(-1, 1))

model = SequentialTFModel(input_dim=2)
model.add(5, activation='relu')
model.add(1, activation='sigmoid')
model.train(X, Y, epochs=100, lr=0.05, loss_fn='bce')

print("Actual:\n", Y.numpy().squeeze())
predictions = model.predict(X)
print("Predictions (rounded):\n", tf.round(predictions).numpy().squeeze())

Epoch 0, Loss: 0.6975
Epoch 10, Loss: 0.4775
Epoch 20, Loss: 0.2015
Epoch 30, Loss: 0.0868
Epoch 40, Loss: 0.0572
Epoch 50, Loss: 0.0446
Epoch 60, Loss: 0.0373
Epoch 70, Loss: 0.0323
Epoch 80, Loss: 0.0283
Epoch 90, Loss: 0.0249
Actual:
 [1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 1.]
Predictions (rounded):
 [1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 1.]
